<a href="https://colab.research.google.com/github/abhinav3632/googleColab/blob/main/Image_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!g++ --version

g++ (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
Copyright (C) 2021 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [30]:
!apt install libopencv-dev
!apt install -qq opencv-doc

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libopencv-dev is already the newest version (4.5.4+dfsg-9ubuntu4+jammy0).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
The following additional packages will be installed:
  fonts-mathjax libjs-mathjax
Suggested packages:
  fonts-mathjax-extras fonts-stix libjs-mathjax-doc
The following NEW packages will be installed:
  fonts-mathjax libjs-mathjax opencv-doc
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 106 MB of archives.
After this operation, 321 MB of additional disk space will be used.
Selecting previously unselected package fonts-mathjax.
(Reading database ... 123598 files and directories currently installed.)
Preparing to unpack .../fonts-mathjax_2.7.9+dfsg-1_all.deb ...
Unpacking fonts-mathjax (2.7.9+dfsg-1) ...
Selecting previously unselected package libjs-mathjax.
Preparing to unpack .../libjs-mathjax_2.7.9+dfsg-1_all.deb ...
Unpacki

In [6]:
%%writefile test_opencv.cpp

#include <opencv2/opencv.hpp>
#include <iostream>

int main() {
    cv::Mat image = cv::Mat::zeros(100, 100, CV_8UC3); // Create a black image
    cv::circle(image, cv::Point(50, 50), 40, cv::Scalar(0, 0, 255), -1); // Draw a red circle

    cv::imwrite("output.png", image); // Save the image
    std::cout << "Image created and saved as output.png" << std::endl;

    return 0;
}

Writing test_opencv.cpp


In [7]:
!g++ -o test_opencv test_opencv.cpp `pkg-config --cflags --libs opencv4`

In [8]:
!./test_opencv

Image created and saved as output.png


In [9]:
%cd /content/Blur_Version_1


/content/Blur_Version_1


In [32]:
%%writefile CMakeLists.txt
cmake_minimum_required(VERSION 3.5)

# Project Name
project(BlurFilters VERSION 1.0)

# C++ Standard
set(CMAKE_CXX_STANDARD 17)
set(CMAKE_CXX_STANDARD_REQUIRED True)

# Find OpenCV Package
find_package(OpenCV REQUIRED)
include_directories(${OpenCV_INCLUDE_DIRS})

# Add the Executable
add_executable(blur_filters Main.cpp Image.cpp Functionality.cpp)

# Link OpenCV libraries
target_link_libraries(blur_filters ${OpenCV_LIBS})


Writing CMakeLists.txt


In [10]:
%%writefile Image.h
#ifndef IMAGE_H
#define IMAGE_H

#include <iostream>
#include <opencv2/opencv.hpp>
#include "EmptyImagePathError.h"
#include "EmptyImageError.h"

class Image
{
private:
    cv::Mat m_src_rgb; // Source image in rgb format

public:
    Image() = default; // Default constructor
    Image(const Image&) = delete; // Deleted copy constructor
    Image& operator=(const Image&) = delete; // Deleted copy assignment
    Image(Image&&) = delete; // Deleted move constructor
    Image& operator=(Image&&) = delete; // Deleted move assignment
    ~Image() = default;

    cv::Mat src_rgb() const { return m_src_rgb;}

    Image(const std::string &imagePath); // Constructor to load image

    void display(const cv::Mat &image, const std::string &windowName) const; // Display image

    static void closeAllWindows(); // Close all openCv windows
};

#endif // IMAGE_H


Writing Image.h


In [11]:
%%writefile Image.cpp
#include "Image.h"

Image::Image(const std::string &imagePath)
{
    if(imagePath.size() == 0 || imagePath == " " || imagePath == ""){
        throw EmptyImagePathError("Error: Invalid Image Path");
    }
    m_src_rgb = cv::imread(imagePath, cv::IMREAD_COLOR);
    if(m_src_rgb.empty()){
        throw EmptyImageError("Error: Unable to load image from given path.");
        exit(1);
    }
}

void Image::display(const cv::Mat &image, const std::string &windowName) const{
    cv::imshow(windowName, image);
}

void Image::closeAllWindows()
{
    cv::waitKey(0);
    cv::destroyAllWindows();
}


Writing Image.cpp


In [12]:
%%writefile Functionality.h
#ifndef FUNCTIONALITY_H
#define FUNCTIONALITY_H

#include "Image.h"
#include "InvalidKernelSizeError.h"
#include "EmptyWindowNameError.h"
#include "FilterType.h"

/*
    A function to apply the boxfilter on the
    given image
    Input: given rgb image, kernel size
    Output: blurred output image
*/

void applyFilter(const Image &img, int kernelSize, FilterType type);


#endif // FUNCTIONALITY_H


Writing Functionality.h


In [13]:
%%writefile Functionality.cpp
#include "Functionality.h"

void applyFilter(const Image &img, int kernelSize, FilterType type)
{
    if(kernelSize % 2 == 0 || kernelSize < 1){
        throw InvalidKernelSizeError("Error: Kernel size must be a positive odd number.");
    }

    cv::Mat result;
    std::string windowName;

    switch (type){
        case FilterType::Box:
            cv::boxFilter(img.src_rgb(), result, -1, cv::Size(kernelSize, kernelSize));
            windowName = "Box Filter";
            break;

        case FilterType::Blur:
            cv::blur(img.src_rgb(), result, cv::Size(kernelSize, kernelSize));
            windowName = "Blur Filter";
            break;

        case FilterType::Gaussian:
            cv::GaussianBlur(img.src_rgb(), result, cv::Size(kernelSize, kernelSize), 0);
            windowName = "Gaussian Filter";
            break;

        case FilterType::Median:
            cv::medianBlur(img.src_rgb(), result, kernelSize);
            windowName = "Median Filter";
            break;

        default:
            throw std::invalid_argument("Error: Invalid Filter Type.");
    }
    if(windowName.size() == 0 || windowName == " " || windowName == ""){
        throw EmptyWindowNameError("Error: Window Name cannot be empty");
    }
    img.display(result, windowName);
}


Writing Functionality.cpp


In [23]:
%%writefile FilterType.h
#ifndef FILTERTYPE_H
#define FILTERTYPE_H

enum class FilterType{
    Box,
    Blur,
    Gaussian,
    Median
};

#endif // FILTERTYPE_H


Writing FilterType.h


In [15]:
%%writefile EmptyImageError.h
#ifndef EMPTYIMAGEERROR_H
#define EMPTYIMAGEERROR_H

#include <stdexcept>
class EmptyImageError : public std::exception
{
private:
    std::string m_msg;

public:
    EmptyImageError(const std::string &msg) : m_msg{msg} {}
    EmptyImageError() = delete;
    EmptyImageError(const EmptyImageError &) = delete;
    EmptyImageError &operator=(const EmptyImageError &) = delete;
    EmptyImageError(EmptyImageError &&) = default;
    EmptyImageError &operator=(EmptyImageError &&) = delete;
    ~EmptyImageError() = default;
    const char *what() const _GLIBCXX_TXN_SAFE_DYN _GLIBCXX_NOTHROW
    {
        return m_msg.c_str();
    }
};

#endif // EMPTYIMAGEERROR_H


Writing EmptyImageError.h


In [16]:
%%writefile EmptyImagePathError.h
#ifndef EMPTYIMAGEPATHERROR_H
#define EMPTYIMAGEPATHERROR_H

#include <stdexcept>
class EmptyImagePathError : public std::exception
{
private:
    std::string m_msg;

public:
    EmptyImagePathError(const std::string &msg) : m_msg{msg} {}
    EmptyImagePathError() = delete;
    EmptyImagePathError(const EmptyImagePathError &) = default;
    EmptyImagePathError &operator=(const EmptyImagePathError &) = delete;
    EmptyImagePathError(EmptyImagePathError &&) = default;
    EmptyImagePathError &operator=(EmptyImagePathError &&) = delete;
    ~EmptyImagePathError() = default;
    const char *what() const _GLIBCXX_TXN_SAFE_DYN _GLIBCXX_NOTHROW
    {
        return m_msg.c_str();
    }
};

#endif // EMPTYIMAGEPATHERROR_H


Writing EmptyImagePathError.h


In [17]:
%%writefile EmptyWindowNameError.h
#ifndef EMPTYWINDOWNAMEERROR_H
#define EMPTYWINDOWNAMEERROR_H

#include <stdexcept>
class EmptyWindowNameError : public std::exception
{
private:
    std::string m_msg;

public:
    EmptyWindowNameError(const std::string &msg) : m_msg{msg} {}
    EmptyWindowNameError() = delete;
    EmptyWindowNameError(const EmptyWindowNameError &) = delete;
    EmptyWindowNameError &operator=(const EmptyWindowNameError &) = delete;
    EmptyWindowNameError(EmptyWindowNameError &&) = default;
    EmptyWindowNameError &operator=(EmptyWindowNameError &&) = delete;
    ~EmptyWindowNameError() = default;
    const char *what() const _GLIBCXX_TXN_SAFE_DYN _GLIBCXX_NOTHROW
    {
        return m_msg.c_str();
    }
};

#endif // EMPTYWINDOWNAMEERROR_H


Writing EmptyWindowNameError.h


In [18]:
%%writefile InvalidKernelSizeError.h
#ifndef INVALIDKERNELSIZEERROR_H
#define INVALIDKERNELSIZEERROR_H

#include <stdexcept>
class InvalidKernelSizeError : public std::exception
{
private:
    std::string m_msg;

public:
    InvalidKernelSizeError(const std::string &msg) : m_msg{msg} {}
    InvalidKernelSizeError() = delete;
    InvalidKernelSizeError(const InvalidKernelSizeError &) = delete;
    InvalidKernelSizeError &operator=(const InvalidKernelSizeError &) = delete;
    InvalidKernelSizeError(InvalidKernelSizeError &&) = default;
    InvalidKernelSizeError &operator=(InvalidKernelSizeError &&) = delete;
    ~InvalidKernelSizeError() = default;
    const char *what() const _GLIBCXX_TXN_SAFE_DYN _GLIBCXX_NOTHROW
    {
        return m_msg.c_str();
    }
};

#endif // INVALIDKERNELSIZEERROR_H


Writing InvalidKernelSizeError.h


In [21]:
%%writefile Main.cpp
#include "Functionality.h"
#include "Time.h"

int main()
{
    std::string imagePath = "/content/Blur_Version_1/lena.png";
    try
    {
        Image img(imagePath);

        img.display(img.src_rgb(), "Original Image");

        int kernelSize;
        std::cout << "Enter the kernel size (odd number):";
        std::cin >> kernelSize;

        {
            Time timer;
            try
            {
                applyFilter(img, kernelSize, FilterType::Box);
                applyFilter(img, kernelSize, FilterType::Blur);
                applyFilter(img, kernelSize, FilterType::Gaussian);
                applyFilter(img, kernelSize, FilterType::Median);
            }
            catch(const InvalidKernelSizeError& e)
            {
                std::cerr << e.what() << '\n';
            }
            catch(const EmptyWindowNameError& e)
            {
                std::cerr << e.what() << '\n';
            }

        }
        img.closeAllWindows();
    }
    catch (const EmptyImagePathError &e)
    {
        std::cerr << e.what() << '\n';
    }
    catch (const EmptyImageError &e)
    {
        std::cerr << e.what() << '\n';
    }
    return EXIT_SUCCESS;
}

Overwriting Main.cpp


In [20]:
%%writefile Time.h
#ifndef TIME_H
#define TIME_H

#include <iostream>
#include <memory>

#include <chrono>

class Time{
    private:
        std::chrono::time_point<std::chrono::high_resolution_clock> m_StartTimePoint;

        void Stop(){
            auto endTimePoint = std::chrono::high_resolution_clock::now();

            auto start = std::chrono::time_point_cast<std::chrono::microseconds>(m_StartTimePoint).time_since_epoch().count();
            auto end = std::chrono::time_point_cast<std::chrono::microseconds>(endTimePoint).time_since_epoch().count();

            auto duration = end - start;
            double ms = duration * 0.001;

            std::cout << "Time Elapsed: " << ms << "ms\n";
        }

    public:
    Time(){
        m_StartTimePoint = std::chrono::high_resolution_clock::now();
    }

    ~Time(){
        Stop();
    }
};

#endif // TIME_H


Writing Time.h


In [33]:
!cmake -B build

-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found OpenCV: /usr (found version "4.5.4")
-- Configuring done (0.9s)
-- Generating done (0.0s)
-- Build files have been written to: /content/Blur_Version_1/build


In [34]:
%cd /content/Blur_Version_1/build

/content/Blur_Version_1/build


In [35]:
!make

[ 25%] Building CXX object CMakeFiles/blur_filters.dir/Main.cpp.o
[ 50%] Building CXX object CMakeFiles/blur_filters.dir/Image.cpp.o
[ 75%] Building CXX object CMakeFiles/blur_filters.dir/Functionality.cpp.o
[100%] Linking CXX executable blur_filters
[100%] Built target blur_filters


In [36]:
!./blur_filters

terminate called after throwing an instance of 'cv::Exception'
  what():  OpenCV(4.5.4) ./modules/highgui/src/window_gtk.cpp:635: error: (-2:Unspecified error) Can't initialize GTK backend in function 'cvInitSystem'

